In [27]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geomloss import SamplesLoss
import torch
from sklearn import preprocessing

In [28]:
gan_swat_network = pd.read_csv("gan_lstm_swat_network.csv", sep=",").to_numpy()
gan_swat_sensor = pd.read_csv("gan_lstm_swat_sensor.csv", sep=",").to_numpy()
gan_wadi = pd.read_csv("gan_lstm_wadi.csv", sep=",").to_numpy()
gan_wustl = pd.read_csv("gan_lstm_wustl.csv", sep=",").to_numpy()

vae_swat_network = pd.read_csv("vae_swat_network.csv", sep=",").to_numpy()
vae_swat_sensor = pd.read_csv("vae_swat_sensor.csv", sep=",").to_numpy()
vae_wadi = pd.read_csv("vae_wadi_sensor.csv", sep=",").to_numpy()
vae_wustl = pd.read_csv("vae_wustl_network.csv", sep=",").to_numpy()

vae_gan_swat_network = pd.read_csv("vae_gan_swat_network.csv", sep=",").to_numpy()
vae_gan_swat_sensor = pd.read_csv("vae_gan_swat_sensor.csv", sep=",").to_numpy()
vae_gan_wadi = pd.read_csv("vae_gan_wadi_sensor.csv", sep=",").to_numpy()
vae_gan_wustl = pd.read_csv("vae_gan_wustl_network.csv", sep=",").to_numpy()

gan_softmax_swat_network = pd.read_csv("gan_softmax_swat_network_50.csv", sep=",").to_numpy()
gan_softmax_swat_sensor = pd.read_csv("gan_softmax_swat_sensor_50.csv", sep=",").to_numpy()
gan_softmax_wadi = pd.read_csv("gan_softmax_wadi_50.csv", sep=",").to_numpy()
gan_softmax_wustl = pd.read_csv("gan_softmax_wustl_50.csv", sep=",").to_numpy()

In [29]:
swat = preprocessing.normalize(pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/swat_processed.csv", sep=",").to_numpy()[:, 1:], axis=0, norm="max")
# swap sport to front
swat[:, [0, 1]] = swat[:, [1, 0]]
# # swap dport to after sport
swat[:, [1, 2]] = swat[:, [2, 1]]
# # swap protocols to after dport
swat[:, [2, 14]] = swat[:, [14, 2]]

wadi = preprocessing.normalize(pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/wadi_attacks.csv", sep=",").to_numpy()[:, 1:])
swat_sensor = preprocessing.normalize(pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/swat_sensor.csv", sep=",").to_numpy()[:, 1:])
wustl = preprocessing.normalize(pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/wustl_attacks.csv", sep=",").to_numpy()[:, 1:])

In [30]:
wadi[np.where(wadi == 0)] = 1e-5
wadi[np.where(wadi < 0)] = 1e-10

In [31]:
def kl(real, fake):

    num_features = real.shape[1]
    real = real[: fake.shape[0]]

    kl_div = np.mean(scipy.special.kl_div(real, fake))

    return kl_div

In [32]:
kl(swat, vae_swat_network[:, 1:]), kl(swat_sensor, vae_swat_sensor), kl(wadi, vae_wadi), kl(wustl, vae_wustl)

(0.13348980234962848,
 0.5979942220894797,
 0.46105836291012164,
 0.1992463569583805)

In [33]:
kl(swat, gan_swat_network[:, 1:]), kl(swat_sensor, gan_swat_sensor), kl(wadi, gan_wadi), kl(wustl, gan_wustl)

(0.20282122166432406,
 0.5122309183283169,
 0.45867170180156547,
 0.2678966469702692)

In [34]:
kl(swat, vae_gan_swat_network[:, 1:]), kl(swat_sensor, vae_gan_swat_sensor.reshape(2640, 28)), kl(wadi, vae_gan_wadi.reshape(1996, 69)), kl(wustl, vae_gan_wustl.reshape(17404, 27))

(0.11842562286893839,
 0.44386005620148394,
 0.46929600781544095,
 0.3485649517470891)

In [35]:
gan_softmax_swat_network[np.where(gan_softmax_swat_network < 0)] = 1e-10
gan_softmax_swat_sensor[np.where(gan_softmax_swat_sensor < 0)] = 1e-10
gan_softmax_swat_sensor[np.where(gan_softmax_swat_sensor == 0)] = 1e-10
gan_softmax_wadi[np.where(gan_softmax_wadi < 0)] = 1e-10
gan_softmax_wadi[np.where(gan_softmax_wadi == 0)] = 1e-10
gan_softmax_wustl[np.where(gan_softmax_wustl < 0)] = 1e-10

In [36]:
kl(swat, preprocessing.normalize(gan_softmax_swat_network, axis=0, norm="max")), kl(swat_sensor, gan_softmax_swat_sensor), kl(wadi, gan_softmax_wadi), kl(wustl, preprocessing.normalize(gan_softmax_wustl, axis=0, norm="max"))

(0.8469856910545747,
 1.0064698457431447,
 0.6145655712144291,
 0.8948334078864799)

In [37]:
np.mean(scipy.special.kl_div(swat_sensor[:2640], vae_swat_sensor))

0.5979942220894797